In [1]:
import urllib
import pandas
import json
import requests
import numpy
import os
import sys
import geopandas
import janitor
from path import Path
import osmnx as ox
import matplotlib.pyplot as plt

In [17]:
import folium

In [188]:
# path configuration
working_directory = Path.getcwd()

inputs_path = working_directory / 'inputs'
outputs_path = working_directory / 'outputs'

In [189]:
# set up api parameters
base_url = "https://safetydata.fra.dot.gov/MasterWebService/PublicApi/frads/v1/odata/gcis"
dataset = 'Crossings'
state_fips = "StateCD eq '29'"
api_key = '7d9f06fbf611c46d2d767ebd72742923'

In [212]:
# Define the number of items to retrieve per API call
items_per_call = 10

# Initialize a variable to keep track of the current skip value
current_skip = 0

total_items_to_retrieve = 100

# Create an empty list to store the data
data_list = []

In [191]:
# Create a loop to make API calls until you reach the total_items_to_retrieve
while current_skip < total_items_to_retrieve:
    # Build the API endpoint with the current skip value
    endpoint = f"{base_url}/{dataset}?&token={api_key}&$format=json&$filter={state_fips}&$skip={current_skip}"

    # Make the API call here
    response = requests.get(endpoint)
    
    # Check if the API call was successful
    if response.status_code == 200:
        # Convert the API response data to a DataFrame
        api_data = response.json().get('value', [])  # Extract 'value' key
        
        # Extend the data_list with the extracted data
        data_list.extend(api_data)
        
        # Increment the skip value for the next iteration
        current_skip += items_per_call
    else:
        print(f"API request failed with status code {response.status_code}")
        break  # Exit the loop if the API request fails

# Create a DataFrame from the data_list
df = pd.DataFrame(data_list)


In [214]:
# by agency ID

agency = 'ReportingAgencyId eq 891'

# Create a loop to make API calls until you reach the total_items_to_retrieve
while current_skip < total_items_to_retrieve:
    # Build the API endpoint with the current skip value
    endpoint = f"{base_url}/{dataset}?&token={api_key}&$format=json&$filter={agency}&$skip={current_skip}"
    # Make the API call here
    response = requests.get(endpoint)
    
    # Check if the API call was successful
    if response.status_code == 200:
        # Convert the API response data to a DataFrame
        api_data = response.json().get('value', [])  # Extract 'value' key
        
        # Extend the data_list with the extracted data
        data_list.extend(api_data)
        
        # Increment the skip value for the next iteration
        current_skip += items_per_call
    else:
        print(f"API request failed with status code {response.status_code}")
        break  # Exit the loop if the API request fails

# Create a DataFrame from the data_list
df = pd.DataFrame(data_list)

In [232]:
df.ReportingAgencyTypeId.unique()

array([1], dtype=int64)

In [230]:
df.columns[df.columns.str.contains('Id')]

Index(['ReportBaseId', 'ReportingAgencyId', 'ReportingAgencyTypeId',
       'ReasonId', 'CrossingId', 'CrossingIdSuffix'],
      dtype='object')

In [219]:
#make points spatial
crs= {'init': 'epsg:6511'}

points = geopandas.GeoDataFrame(
    df, crs=crs, geometry=geopandas.points_from_xy(df.Longitude, df.Latitude)
)

C:\Users\nanzawi\AppData\Local\anaconda3\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [220]:
# Latitude and Longitude coordinates for Missouri
missouri_coordinates = [38.573936, -92.603760]

# Create a Folium map centered around Missouri
m = folium.Map(location=missouri_coordinates, zoom_start=7)

In [221]:
points[['Latitude','Longitude']] = points[['Latitude','Longitude']].astype(float)

In [222]:
# Assuming you have a GeoDataFrame called "points" with a "geometry" column containing Point geometries
for idx, row in points.iterrows():
    # Get the coordinates of the point
    lon, lat = row.geometry.coords[0]

    # Create a marker for each point and add it to the map
    folium.Marker([lat, lon], icon=folium.Icon(icon="cloud")).add_to(m)

    
# Save the map to an HTML file or display it in a Jupyter Notebook
m